In [4]:
%load_ext autoreload
import glob
import nibabel
import pandas as pd
import numpy as np
import h5py
from collections import defaultdict, Counter
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed

from IPython.core.debugger import set_trace

import os
import shutil
import pickle

import torch
import torch.nn.functional as F

from utils import create_dicts, normalize, load, video, video_comparison

from multiprocessing import cpu_count
N_CPU = cpu_count()

from IPython.core.display import display, HTML
from celluloid import Camera

SEED = 42
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Affine transform

In [3]:
affine_pth = '../fcd_data/radiologist/fmriprep/sub-43/anat/sub-43_acq-T1MprageSagP2Iso_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'

In [5]:
matrix = h5py.File(affine_pth, "r")

In [15]:
matrix_tg = matrix['TransformGroup']

In [29]:
for k,v in matrix_tg.items():
    print('----------------')
    print(k,v, 'has:...')
    
    for k_,v_ in v.items():
        print(k_,v_)
        

----------------
0 <HDF5 group "/TransformGroup/0" (1 members)> has:...
TransformType <HDF5 dataset "TransformType": shape (1,), type "|O">
----------------
1 <HDF5 group "/TransformGroup/1" (3 members)> has:...
TransformFixedParameters <HDF5 dataset "TransformFixedParameters": shape (3,), type "<f8">
TransformParameters <HDF5 dataset "TransformParameters": shape (12,), type "<f4">
TransformType <HDF5 dataset "TransformType": shape (1,), type "|O">
----------------
2 <HDF5 group "/TransformGroup/2" (3 members)> has:...
TransformFixedParameters <HDF5 dataset "TransformFixedParameters": shape (18,), type "<f8">
TransformParameters <HDF5 dataset "TransformParameters": shape (25590063,), type "<f4">
TransformType <HDF5 dataset "TransformType": shape (1,), type "|O">


In [6]:
matrix.create_dataset()

<HDF5 file "sub-43_acq-T1MprageSagP2Iso_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5" (mode r)>

In [6]:
brain_path_orig = '/nfs/garfield/ibulygin/preprocessed_data_v2/fmriprep/sub-n8/anat/'
brain_path_mn152 = '/nfs/garfield/ibulygin/preprocessed_data_v2/fmriprep/sub-n10/anat/sub-n10_acq-T1MprageSagP2IsoOrig_desc-preproc_T1w.nii.gz'

brain_sample = nibabel.load(brain_path).get_fdata()

In [ ]:
ls -la 

In [4]:
brain_sample.shape

(192, 352, 352)

# Loading data

In [2]:
labels_components = np.load('label_info_v2.npy', allow_pickle=True).item()
single_component_keys = {k for k,v in labels_components.items() if len(v['cc3d'][0]) != 2}

labels_broken = {'43','n27','33','37', '9', 'n31',
                 'n30', 'n10', 'n8', '47', '6',
                 '25', '12'}.union(single_component_keys)

In [3]:
len(labels_broken)

17

In [4]:
root = '/nfs/garfield/ibulygin/preprocessed_data_v2'
root_label = os.path.join(root, 'label')


feature_paths_templates = {
      'label':os.path.join(root, 'label/{label}.nii'),
      'mask':os.path.join(root, 'fmriprep/sub-{label}/anat/*MNI152NLin2009cAsym_desc-brain_mask.nii.gz'),
      'curv':os.path.join(root, 'curv/norm-{label}.nii'),
      'sulc':os.path.join(root, 'sulc/norm-{label}.nii'),
      'thickness':os.path.join(root, 'thickness/norm-{label}.nii'),
      'bT1':os.path.join(root, 'bluring-T1/sub-{label}_T1blurring-in-volume.nii.gz'),
      'bT2':os.path.join(root, 'bluring-T2/sub-{label}_T2blurring-in-volume.nii.gz'),
      'T1w':os.path.join(root, 'fmriprep/sub-{label}/anat/*MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'),
      'T2':os.path.join(root, 'T2/{label}.nii'),
      'CSF':os.path.join(root, 'fmriprep/sub-{label}/anat/*MNI152NLin2009cAsym_label-CSF_probseg.nii.gz'),
      'GM':os.path.join(root, 'fmriprep/sub-{label}/anat/*MNI152NLin2009cAsym_label-GM_probseg.nii.gz'),
      'WM':os.path.join(root, 'fmriprep/sub-{label}/anat/*MNI152NLin2009cAsym_label-WM_probseg.nii.gz'),
        }

paths_dict = create_dicts(root_label,
                         feature_paths_templates, 
                         broken_labels=labels_broken)

No curv for 50
No curv for 59
No curv for 57
No curv for 52
No bT1 for 27
No curv for 53
No curv for 51
No curv for 58
No curv for n20
No curv for n23
No curv for n25
No curv for G015


In [45]:
# ls /nfs/garfield/ibulygin/tensors_v2/ | wc -l

In [5]:
len(paths_dict)

63

In [27]:
def make_tensor(root, name, pdict):
    
    data_dict = load(pdict)
    
    ################
    # MAKE TENSORS #
    ################
    
    mask = data_dict['mask'] > 0 # it should be bool!

    label = data_dict['label'] * mask.astype(int)
    label = np.array(label) > 0
    
    mask_tensor_torch = torch.tensor(mask, dtype=torch.float32)
    label_tensor_torch = torch.tensor(label, dtype=torch.float32)
    
    torch_tensor = {'mask':mask_tensor_torch,
                    'label':label_tensor_torch}
    
    for k,v in data_dict.items():
        if not k in ['mask', 'label']:
            data = normalize(v, mask)
            data_torch = torch.tensor(data, dtype=torch.float32)
            torch_tensor[k] = data_torch
    
    torch.save(torch_tensor, os.path.join(root, f'tensor_{name}'))
#     return torch_tensor

In [28]:
# %%time
# tensor = make_tensor('./', '44', paths_dict['44'])

In [29]:
# for k,v in tensor.items():
#     print(k,v.shape, v.max(), v.min())

In [30]:
# brain = tensor['T1w'].numpy()

In [31]:
# plt.ioff()
# camera = video_comparison([t.numpy() for t in tensor.values()],
#                           n_slides=128)
# animation = camera.animate() # animation ready
# HTML(animation.to_html5_video()) # displaying the animation

In [32]:
# plt.figure()
# plt.imshow(brain[:,64,:])
# plt.show()

In [33]:
tensors_root = '/nfs/garfield/ibulygin/tensors_v2'

In [34]:
_ = Parallel(n_jobs=N_CPU//2)(delayed(make_tensor)(tensors_root, 
                                             k, v) \
                        for k,v in tqdm(paths_dict.items()))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [10:07<00:00,  9.64s/it]


In [46]:
tensor = torch.load(os.path.join(tensors_root, 'tensor_44'))

In [47]:
for k,v in tensor.items():
    print(k, v.max(), v.min(), v.shape)

mask tensor(1.) tensor(0.) torch.Size([241, 336, 283])
label tensor(1.) tensor(0.) torch.Size([241, 336, 283])
curv tensor(1.) tensor(0.) torch.Size([241, 336, 283])
sulc tensor(1.) tensor(0.) torch.Size([241, 336, 283])
thickness tensor(1.) tensor(0.) torch.Size([241, 336, 283])
bT1 tensor(1.) tensor(0.) torch.Size([241, 336, 283])
bT2 tensor(1.) tensor(0.) torch.Size([241, 336, 283])
T1w tensor(1.) tensor(0.) torch.Size([241, 336, 283])
T2 tensor(1.) tensor(0.) torch.Size([241, 336, 283])
CSF tensor(1.) tensor(0.) torch.Size([241, 336, 283])
GM tensor(1.) tensor(0.) torch.Size([241, 336, 283])
WM tensor(1.) tensor(0.) torch.Size([241, 336, 283])


# Metadata

In [48]:
metadata_path = '/nfs/garfield/ibulygin/metadata_v2'
train_keys, test_keys = train_test_split(list(paths_dict.keys()), test_size=0.1, random_state=SEED)
metadata = {'train':train_keys,
            'test':test_keys,
            'seed':SEED}

np.save(metadata_path, metadata)

In [49]:
metadata = np.load(f'{metadata_path}.npy',allow_pickle=True).item()

In [50]:
len(metadata['train'])

56

# Yarkin data (draft)

In [14]:
# root = '../fcd_data/yarkin_06dec2021'

In [16]:
# ls -la ../fcd_data/yarkin_06dec2021/sub-9/ # sub-17_MNI152NLint2_brain-Flair.nii

total 1175352
drwxr-xr-x  2 ibulygin users      4096 дек  6 15:33 ./
drwxrwxrwx 92 ibulygin users      4096 дек  6 15:47 ../
-rw-r--r--  1 ibulygin users 183330016 дек  6 15:33 c1sub-9_MNI152NLint2_struct-Flair.nii
-rw-r--r--  1 ibulygin users  45613408 дек  6 15:33 c1sub-9_struct-Flair.nii
-rw-r--r--  1 ibulygin users  45613408 дек  6 15:33 c2sub-9_struct-Flair.nii
-rw-r--r--  1 ibulygin users  45613408 дек  6 15:33 c3sub-9_struct-Flair.nii
-rw-r--r--  1 ibulygin users  91226464 дек  6 15:33 hrsub-9_brain-Flair_mask.nii
-rw-r--r--  1 ibulygin users    256056 дек  6 15:33 hrsub-9_brain-Flair_mask.nii.gz
-rw-r--r--  1 ibulygin users 182452576 дек  6 15:33 hrsub-9_brain-Flair.nii
-rw-r--r--  1 ibulygin users  21892116 дек  6 15:33 hrsub-9_brain-Flair.nii.gz
-rw-r--r--  1 ibulygin users 182452576 дек  6 15:33 msub-9_struct-Flair.nii
-rw-r--r--  1 ibulygin users   2371932 дек  6 15:33 norm_and_wgsub-9_struct-Flair.nii
-rw-r--r--  1 ibulygin users   2371932 дек  6 15:33 skullstriped-sub-9_s

c1sub-9_MNI152NLint2_struct-Flair.nii - **asym**  
c1sub-9_struct-Flair.nii - **SYM!**  
c2sub-9_struct-Flair.nii - **SYM!**  
c3sub-9_struct-Flair.nii - **SYM!**  
hrsub-9_brain-Flair_mask.nii - **SYM!**  
hrsub-9_brain-Flair_mask.nii.gz - **SYM!**  
hrsub-9_brain-Flair.nii - **checK IT!! SYM!**  
hrsub-9_brain-Flair.nii.gz - **SYM!**  
msub-9_struct-Flair.nii - **SYM!, tissue**  
norm_and_wgsub-9_struct-Flair.nii - **SYM, lowdim**  
skullstriped-sub-9_struct-Flair.nii - **SYM, lowdim**  
sub-9_MNI152NLint2_brain-Flair.nii - **BRAIN, 1X1, ASYM**    
sub-9_struct-Flair.nii - **ASYM**  
sub-9_struct-Flair_seg_inv_sn.mat  
sub-9_struct-Flair_seg_sn.mat  
wc1_or_wc2sub-9_struct-Flair.nii - **SYM lowdim**  
wc1sub-9_struct-Flair.nii - **SYM lowdim**  
wc2sub-9_struct-Flair.nii - **SYM lowdim**    
wc3sub-9_struct-Flair.nii - **SYM lowdim**    
wmsub-9_struct-Flair.nii - **SYM lowdim tissues**   

In [82]:
# brain = nibabel.load('../fcd_data/yarkin_06dec2021/sub-9/sub-9_MNI152NLint2_brain-Flair.nii').get_fdata()
# brain = nibabel.load('../fcd_data/yarkin_06dec2021/sub-9/hrsub-9_brain-Flair.nii').get_fdata()